In [ ]:
# here is the code for generating the feature and label matrices!

# IMPORTING THE BACKGROUND STUFF
import cv2
import numpy as np
import os, math, csv
from keras.models import Model
from keras.applications.resnet50 import ResNet50

# A helper function for later down the road --> for converting fear annotations to one-hot vectors
def fear_oneHot(movie_length, fear_path):
    # load start and end times from file
    y_data = np.loadtxt(fear_path, skiprows=1)
    y_data_input = np.zeros((movie_length))
    
    # need to address corner case if y_data is 1D array
    if type(y_data[0]) == np.float64:
        y_data_ext = np.zeros([1,2])
        y_data_ext[:,:] = np.asarray(y_data)
    else:
        y_data_ext = y_data 
        
          
    # for each element in first dimension of the y_data array
    for i in range(y_data_ext.shape[0]):
        # access the start time number and end time number
        start = int(y_data_ext[i][0])
        end = int(y_data_ext[i][1])
        # set the elements between these indices in the zeros array to one
        y_data_input[start] = 1 #maybe superfluous
        y_data_input[end] = 1
        y_data_input[start:end] = 1
        
    return y_data_input

In [ ]:
# FOR LOADING THE VGG16 DATA

VGG_data_dim = 4096
timesteps= 5564 # the max number of timesteps --> note that the movies have different lengths
num_movies = #TODO - THE NUMBER OF MOVIES IN THE FOLDER NEEDS TO BE CONFIRMED FOR EACH DEVSET
VGG16_feature_data = np.zeros([num_movies, timesteps, VGG_data_dim])

# for each movie
for num in range(num_movies):
    # load the X_input data
    # get the file path
    mov = 54 + num # TODO - MAY BE NECESSARY TO ADD A CONSTANT IN ORDER TO GET CORRECT MOVIE NUMBER
    # TODO - PLEASE CONFIRM FILE PATH
    path = os.path.join('/Users/chloeloughridge/Desktop/MEDIAEVAL18-TestSet-Visual_features/visual_features/MEDIAEVAL18_{}/fc6'.format(mov))
    count = 0
    # loading the VGG16 feature data from each file in the folder
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        if filename.endswith(".txt"):
            input_data = np.loadtxt(os.path.join(path, file), delimiter=',')
            VGG16_feature_data[num, count, :] = np.asarray(input_data)[:]
            count = count + 1
            continue
        else:
            continue
    print(num)
            
print(count)
print(VGG16_feature_data.shape)
print(VGG16_feature_data)

In [ ]:
# SAVING THE VGG16 FEATURE DATA

file_name = None # TODO - Maybe something like VGG_feature_data01 for devset part 1?
path = None # TODO - path for saving the file
np.save(file_name, path)

In [ ]:
# FOR CREATING THE LABELS MATRIX

# please download Fear Annotations from Liris Accede dataset

labels_matrix = np.zeros([num_movies, timesteps])

for num in range(num_movies):
    # create the appropriate path to the fear annotation data
    # TODO - PLEASE CONFIRM THESE ARE THE CORRECT PATHS
    if num < 10:
        path = os.path.join('/Users/chloeloughridge/Desktop/AI Grant Project/Dataset/2018 Task Data/MEDIAEVAL18-DevSet-Part1-Fear-annotations/annotations/MEDIAEVAL18_0{}_Fear.txt'.format(num))
    if num > 10:
        path = os.path.join('/Users/chloeloughridge/Desktop/AI Grant Project/Dataset/2018 Task Data/MEDIAEVAL18-DevSet-Part1-Fear-annotations/annotations/MEDIAEVAL18_{}_Fear.txt'.format(num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    labels_matrix[num, :] = y_data

In [ ]:
# SAVING THE LABELS MATRIX

file_name = None # TODO - Maybe something like labels01 for devset part 1?
path = None # TODO - path for saving the file
np.save(file_name, path)

In [ ]:
# GENERATING RESNET50 FEATURE DATA

# The following code assumes that 1 frame per second from each movie has been saved to a folder on Desktop
# called "frames". Within "frames" I created a numbered folder for each movie.

# I ran the following terminal command for each movie to extract one frame per second
# the downside is that I then had to rerun this command for each movie in the dataset (and tweak the destination path
# accordingly). Perhaps there is a better way?

# Here is the command: 
# ffmpeg -loglevel error -i [MOVIE NAME].mp4 -r 1 -f [DESTINATION PATH] frame-%05d.jpg

In [ ]:
# FOR GENERATING RESNET50 FEATURE DATA

# import pretrained InceptionV3 model from keras
model = InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
print(model.summary())

In [ ]:
# GENERATING RESNET50 FEATURE DATA CONTINUED
# iterate though each movie in the folder

num_movies = # TODO - however many movies are in this section of the dataset

# set up master array
inception_v3_feature_data = np.zeros([num_movies, timesteps, 248832]) # TODO - this last number may be wrong; keras 
# will probably throw an enlightening error if this is the case

for num in range(num_movies):
    count = 0
    for file in os.listdir('../Desktop/frames/{}/'.format(num)):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            # extract the image data
            test_img = cv2.imread(os.path.join('../Desktop/frames/{}/'.format(num), filename))
            data = (test_img/255)[np.newaxis, :, :, :]
            # send image data through the resnet model
            intermediate_output = model.predict(data)
            # reshape model output
            feature_data = intermediate_output.flatten()
            # slot into the master away
            inception_v3_feature_data[num, count, :] = feature_data
        
            count = count + 1
            continue
        else:
            continue
    print(num)

In [ ]:
# SAVE THE RESNET50 FEATURE DATA
filename = #TODO
path = # TODO
np.save(filename, path)